In [35]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

In [36]:
#Create app
app = dash.Dash(__name__)

In [37]:
# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [38]:
# Read the wildfire data into pandas dataframe
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

In [39]:
df.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced
0,NSW,1/4/2005,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R
1,NSW,1/5/2005,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R
2,NSW,1/6/2005,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R
3,NSW,1/7/2005,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R
4,NSW,1/8/2005,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R


In [40]:
#Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name() #used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

In [41]:
#Two extra columns are added at the end i.e, Month and Year
df.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,Month,Year
0,NSW,1/4/2005,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R,January,2005
1,NSW,1/5/2005,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R,January,2005
2,NSW,1/6/2005,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R,January,2005
3,NSW,1/7/2005,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R,January,2005
4,NSW,1/8/2005,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R,January,2005


Layout Section of Dash

In [42]:
#Application Layout
app.layout = html.Div(
    children=[
        html.H1("Fire Dashboard"),
        
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Label("Select Year"),
                        dcc.Dropdown(
                            id='year-dropdown',
                            options=[
                                {'label': year, 'value': year}
                                for year in df['Year'].unique()
                            ],
                            value=df['Year'].max(),  # Set default value to the latest year
                            multi=False
                        ),
                        
                        html.Label("Select Region"),
                        dcc.RadioItems(
                            id='region-radio',
                            options=[
                                {'label': region, 'value': region}
                                for region in df['Region'].unique()
                            ],
                            value=df['Region'].unique()[0],  # Set default value to the first region
                        ),
                    ],
                    style={'width': '48%', 'display': 'inline-block'}
                ),
                
                html.Div(
                    children=[
                        dcc.Graph(id='pie-chart'),
                        dcc.Graph(id='bar-chart')
                    ],
                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}
                ),
            ]
        )
    ]
)

In [43]:
# Callback to update pie chart
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('year-dropdown', 'value'),
    Input('region-radio', 'value')]
)

def update_pie_chart(selected_year, selected_region):
    filtered_df = df[(df['Year'] == selected_year) & (df['Region'] == selected_region)]
    fig = px.pie(
        filtered_df,
        names='Month',
        values='Estimated_fire_area',
        title=f'Monthly Average Estimated Fire Area ({selected_region} - {selected_year})'
    )
    return fig

In [44]:
# Callback to update bar chart
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('region-radio', 'value')]
)

def update_bar_chart(selected_year, selected_region):
    filtered_df = df[(df['Year'] == selected_year) & (df['Region'] == selected_region)]
    fig = px.bar(
        filtered_df,
        x='Month',
        y= 'Count',
        title=f'Monthly Average Count of Pixels for Presumed Vegetation Fires ({selected_region} - {selected_year})'
    )
    return fig

In [45]:
app.run_server(debug=True)